# CTR: Integrity issues

Encrypt the contents of receipt.txt using AES-256-CTR, with a key and IV of your choosing. Calculate the size of the header up through the identification of the recipient and extract that many bytes of the encrypted data.

XOR the plaintext header with the ciphertext header to get the keystream. Once you have this, XOR the extracted keystream with a header identifying Elon Tusk as the recipient.
Copy the new header ciphertext over the bytes of the encrypted file to create a new message identifying Elon Tusk a the recipient of the money.

Now repeat the same process, changing the amount of money the recipient should receive to $9.999.999
In the end you should have a ciphertext which when decrypted will acknowledge that Elon Tusk should receive $9.999.999.

Prove that it works by printing in the console the original and modified ciphertexts as well as the modified message

In [ ]:

import secrets
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes



In [ ]:

def xor(a: bytes, b: bytes):
    return bytes([char_a ^ char_b for char_a, char_b in zip(a, b)])


def encrypt_CTR(plain: bytes, key: bytes, nonce: bytes):
    cipher = Cipher(algorithms.AES256(key), modes.CTR(nonce))
    encryptor = cipher.encryptor()
    return encryptor.update(plain) + encryptor.finalize()


def corrupt(plain: bytes, cipher: bytes):
    keystream = xor(plain, cipher)
    cipher = bytearray(cipher)

    cipher[39] = keystream[39] ^ ord('T')
    cipher[99:108] = xor(keystream[99:108], b'9.999.999')

    return bytes(cipher)


def decrypt_CTR(ciphertext: bytes, key: bytes, nonce: bytes):
    cipher = Cipher(algorithms.AES256(key), modes.CTR(nonce))
    decryptor = cipher.decryptor()
    return decryptor.update(ciphertext) + decryptor.finalize()


In [ ]:

def main(receipt):
    
    
    plain = receipt.encode()

    key = secrets.token_bytes(32)
    nonce = secrets.token_bytes(16)

    cipher = encrypt_CTR(plain, key, nonce)
    corrupted = corrupt(plain, cipher)

    decrypted = decrypt_CTR(corrupted, key, nonce)
    print(f"Key: {key.hex()}")
    print(f"Nonce: {nonce.hex()}")

    print(f"Original ciphertext: {cipher.hex()}")
    print(f"Generated ciphertext: {corrupted.hex()}")
    print(f"Generated plaintext: ")
    print(decrypted.decode())




In [ ]:
if __name__ == "__main__":

    reciept = "CreditCardPurchase  Recipient: Elon Musk Buyer: Jeff Bezos Date: 02/04/2020 Amount:$1.000.000  CCNumber: 555-555-555-555"

    main(reciept)